## Prepare data for synthetic 3dshape dataset

In [1]:
from tqdm import tqdm
import time
import argparse
import os
import sys
from abc import abstractmethod
from torch.nn import functional as F

In [2]:
import pandas as pd
import numpy as np
import einops

In [3]:
import torch
from torch import nn
import torch.nn.init as init
from torchvision import datasets
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader

In [4]:
sys.path.insert(0,"/nfshomes/xliu1231/Causal_Disentangle/3dshape_dataset")

In [5]:
from datasets import ShapeDataset

In [6]:
from dataset_3d_shape import sample_3dshape_dataset

In [7]:
from vae import VAE

### Prepare dataset

_FACTORS_IN_ORDER = ['floor_hue', 'wall_hue', 'object_hue', 'scale', 'shape', 'orientation']


_NUM_VALUES_PER_FACTOR = {'floor_hue': 10, 'wall_hue': 10, 'object_hue': 10, 'scale': 8, 'shape': 4, 'orientation': 15}


In [8]:
#Set distributions of factors' values. If not specified, we will sample factors randomly

dist={'shape': [1, 0, 0, 0],
      'object_hue': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
     }

In [ ]:
#sample a dataset and save

sample_3dshape_dataset(dist, data_size=1000, 
                       data_path='/nfshomes/xliu1231/Causal_Disentangle/data',
                       label_path='/nfshomes/xliu1231/Causal_Disentangle/data/sample_source.csv',
                       new_label=["SO12", 1]
                      )

In [ ]:
dist={'shape': [1, 0, 0, 0],
      'object_hue': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
     }

sample_3dshape_dataset(dist, data_size=1000, 
                       data_path='/nfshomes/xliu1231/Causal_Disentangle/data',
                       label_path='/nfshomes/xliu1231/Causal_Disentangle/data/sample_source.csv',
                       new_label=["SO12", 1]
                      )

In [ ]:
dist={'shape': [1, 0, 0, 0],
      'object_hue': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
     }

sample_3dshape_dataset(dist, data_size=1000, 
                       data_path='/nfshomes/xliu1231/Causal_Disentangle/data',
                       label_path='/nfshomes/xliu1231/Causal_Disentangle/data/sample_source.csv',
                       new_label=["SO12", 1]
                      )

In [ ]:
transform = T.ToTensor()

In [ ]:
# _FACTORS_IN_ORDER = ['floor_hue', 'wall_hue', 'object_hue', 'scale', 'shape', 'orientation']
# concept_interest is the indices of the interested factors that will be labels

train_dataset = ShapeDataset( data_path='/nfshomes/xliu1231/Causal_Disentangle/data', 
                              attr_path='/nfshomes/xliu1231/Causal_Disentangle/data/sample_target.csv',
                              attr=['floor_hue', 'shape'],
                              transform=transform
                             )

In [ ]:
train_loader = torch.utils.data.DataLoader(
                        train_dataset, 
                        batch_size=4, 
                        shuffle=True, 
                        pin_memory=True,
                        num_workers=1)

In [ ]:
prefetcher = data_prefetcher(train_loader)

In [ ]:
i = 0
for epoch in range(1, 30):
    
    #model.train()
    
    i += 1
    
    for input, y in train_loader:
        input = input.cuda()
        print(y)
        continue
        output = model(input)
        recons_loss = F.mse_loss(output[0], input)
        mu = output[1]
        log_var = output[2]
        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
        kld_weight = input.shape[0] / ( 64 * len(train_loader) )
        loss = recons_loss + kld_loss * kld_weight
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    break
        
        

        

---